In [1]:
# the notebook will reload external python modules;
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../codes')

In [3]:
from yolov3 import darknet

ModuleNotFoundError: No module named 'util'

In [ ]:
blocks = darknet.parse_cfg("../codes/yolov3/cfg/yolov3.cfg")

In [ ]:
print(darknet.create_modules(blocks))

## 

In [ ]:
def get_test_input(input_path):
    img = cv2.imread(input_path)
    img = cv2.resize(img, (416,416))          #Resize to the input dimension
    img_ =  img[:,:,::-1].transpose((2,0,1))  # BGR -> RGB | H X W C -> C X H X W 
    img_ = img_[np.newaxis,:,:,:]/255.0       #Add a channel at 0 (for batch) | Normalise
    img_ = torch.from_numpy(img_).float()     #Convert to float
    img_ = Variable(img_)                     # Convert to Variable
    return img_

In [ ]:
"dog-cycle-car.png"